In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc 
import sympy as sy 

# Traffic Pattern

使用流量矩阵$\lambda$来描述：（个人认为类似于连续马尔可夫过程的转移矩阵的感觉）


In [3]:
N = 5

## 随机流量：到各个节点的流量都相同

$\lambda_{sd}=1/N$ 实际在做的时候我是 $1 / (N - 1)$ 因为对角线元素没有用

In [33]:
L = np.ones((N, N)) * (1 / (N - 1)) 
print(L)

[[0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]]


## 排列流量：每一个节点发送给固定的节点

d = $\pi (s)$

In [8]:
import numpy as np

# 定义源地址
source = np.array([0, 1, 2, 3])

# 定义位排列函数
def bit_permutation(source, f, g):
    return np.bitwise_xor(f(source), g(source))

# 定义位补码函数
def bit_complement(source):
    return ~source

# 定义位反转函数
def bit_reverse(source, b):
    return source[::-1]

# 定义位旋转函数
def bit_rotation(source, b):
    return np.mod(source + 1, b)

# 定义洗牌函数
def shuffle(source, b):
    return np.mod(source - 1, b)

# 定义转置函数
def transpose(source, b):
    return np.mod(source + b//2, b)

# 测试这些函数
print("Bit permutation: ", bit_permutation(source, lambda x: x, lambda x: x))
print("Bit complement: ", bit_complement(source))
print("Bit reverse: ", bit_reverse(source, 4))
print("Bit rotation: ", bit_rotation(source, 4))
print("Shuffle: ", shuffle(source, 4))
print("Transpose: ", transpose(source, 4))


Bit permutation:  [0 0 0 0]
Bit complement:  [-1 -2 -3 -4]
Bit reverse:  [3 2 1 0]
Bit rotation:  [1 2 3 0]
Shuffle:  [3 0 1 2]
Transpose:  [2 3 0 1]


# 利用多商品流问题的变体来求解理想最大吞吐量

通过修改多商品流问题的需求满足约束，将需求从恒定值改为可变值，然后目标函数即最大化需求，通过观察需求值的最大值来判断网络在某一种流量模式下的理想最大吞吐

## 多商品流问题的约束条件如下：

1. **容量限制**：对于每一条边(u, v)，所有商品的流量之和不能超过这条边的容量。用数学公式表示就是：
   $$\sum_{i=1}^{k}f_{i}(u,v)\leq c(u,v)$$
   这里，$f_{i}(u,v)$表示商品$i$从节点$u$到节点$v$的流量，$c(u,v)$表示边(u, v)的容量。

2. **流守恒**：对于每一个商品$i$和每一个非源非汇节点$u$，进入节点$u$的流量之和等于离开节点$u$的流量之和。用数学公式表示就是：
   $$\sum_{q\in V}f_{i}(u,q)-\sum_{w\in V}f_{i}(w,u)=0\quad \mathrm {when} \quad u\neq s_{i},t_{i}$$
   这里，$s_{i}$和$t_{i}$分别表示商品$i$的源节点和汇节点。

3. **需求的满足**：对于每一个商品$i$，从源节点$s_{i}$出发的流量之和等于进入汇节点$t_{i}$的流量之和，也就是商品$i$的需求量$d_{i}$。用数学公式表示就是：
   $$\sum_{w\in V}f_{i}(s_{i},w)=d_{i}\Leftrightarrow \sum_{w\in V}f_{i}(w,t_{i})=d_{i}$$

这些约束条件确保了每一个商品的流量在网络中的传输不会超过任何一条边的容量，同时也满足了每一个商品的需求量。

## 问题对求解最大吞吐的适配
这里为了求解最大吞吐，我修改了这个约束条件，使得需求成为我要寻找的决策变量，即：

首先使用流量矩阵 $L$ 来表示一种特定的流量模式，即上文提到的流量模式，然后使用描述吞吐量指标的$d$，他们的含义是：

L的每一个非对角线元素则为，假设节点i传输1单位流量，则$L_{ij}$为节点i传输到j的流量，d则代表节点i将会传输d单位的流量，所以实际的流量传输为$dL$

1. **容量限制**：对于每一条边(u, v)，所有传输流量之和不能超过这条边的容量。用数学公式表示就是：
   $$\sum_{i=1}^{k}f_{i}(u,v)\leq c(u,v)$$
   这里，$f_{i}(u,v)$表示商品$i$从节点$u$到节点$v$的流量，$c(u,v)$表示边(u, v)的容量。

2. **流守恒**：对于每一个传输$i$和每一个非源非汇节点$u$，进入节点$u$的流量之和等于离开节点$u$的流量之和。用数学公式表示就是：
   $$\sum_{q\in V}f_{i}(u,q)-\sum_{w\in V}f_{i}(w,u)=0\quad \mathrm {when} \quad u\neq s_{i},t_{i}$$
   这里，$s_{i}$和$t_{i}$分别表示商品$i$的源节点和汇节点。

3. **需求的满足**：对于每一个传输$i$，从源节点$s_{i}$出发的流量之和等于进入汇节点$t_{i}$的流量之和，也就是转移矩阵中每一个非对角线元素，记为$L_{s_i t_i}$的值乘以吞吐指标$d$。用数学公式表示就是：
   $$\sum_{w\in V}f_{i}(s_{i},w)=d * L_{s_i t_i}\Leftrightarrow \sum_{w\in V}f_{i}(w,t_{i})= d * L_{s_i t_i}$$

而目标函数就是: 
$$\max d$$


目前求解结果如下：
- 对于 2D-mesh 网络，理想最大吞吐不超过2
- 对于 2D-torus 网络，理想最大吞吐不超过4，并随网络规模增大而逐渐递减

In [2]:
import math
import random
import cvxpy as cp
import numpy as np

def get_ring(size):
    e = []

def get_torus(size):
    # 构造网络边
    e = []
    for i in range(size):
        for j in range(size):
            loc = lambda i, j : i * size + j
            left = loc((i - 1 + size) % size, j)
            right = loc((i + 1) % size, j)
            up = loc(i, (j + 1) % size)
            down = loc(i, (j - 1 + size) % size)
            this = loc(i, j)
            to = [left, right, up, down]
            for y in to:
                if y is None:
                    continue
                e.append((this, y, 1.0))
    return e

def get_mesh(size):
    # 构造网络边
    e = []
    for i in range(size):
        for j in range(size):
            loc = lambda i, j : i * size + j
            left = loc(i - 1, j) if i - 1 >= 0 else None
            right = loc(i + 1, j) if i + 1 < size else None 
            up = loc(i, j - 1) if j - 1 >= 0 else None 
            down = loc(i, j + 1) if j + 1 < size else None
            this = loc(i, j)
            to = [left, right, up, down]
            for y in to:
                if y is None:
                    continue
                e.append((this, y, 1.0))
    return e

def get_random_traffic_pattern(size):
    # 初始化流量矩阵
    L = np.ones((size, size)) * (1 / (size - 1))
    return L

def get_traffic_pattern_c(L, size):
    # 构造流量模式
    c = []
    for i in range(size):
        for j in range(size):
            if i != j:
                # 每个元素是一个元组，包含源节点、目标节点和需求量
                c.append((i, j, L[i][j]))
    return c


def get_result(n, e, c):
    k = len(c)
    # 初始化变量
    f = [[[None for _ in range(n)] for _ in range(n)] for _ in range(k)]
    for u, v, w in e:
        for i in range(k):
            f[i][u][v] = cp.Variable(nonneg=True)
    throughput = cp.Variable(nonneg=True)

    def adv_sum(expr):
        return sum([x for x in expr if x is not None])
    # 定义约束
    constraints = []

    # 容量限制
    real_capacity = [[None for _ in range(n)] for _ in range(n)]
    for i in range(len(e)):
        u, v, capacity = e[i]
        real_capacity[u][v] = capacity
    for i in range(n):
        for j in range(n):
            if real_capacity[i][j] is not None:
                constraints.append(adv_sum([f[x][i][j] for x in range(k)]) <= real_capacity[i][j])

    # 流守恒
    for i in range(k):
        for u in range(n):
            s, t, d = c[i]
            if u != s and u != t:
                constraints.append(adv_sum([f[i][u][v] for v in range(n)]) - adv_sum(f[i][v][u] for v in range(n)) == 0)

    # 需求的满足
    for i in range(k):
        s, t, d = c[i]
        constraints.append(adv_sum([f[i][s][v] for v in range(n)]) == d * throughput)
        constraints.append(adv_sum([f[i][v][t] for v in range(n)]) == d * throughput)

    # 定义目标函数()
    obj = cp.Maximize(throughput)

    # 构造问题
    prob = cp.Problem(obj, constraints)

    # 求解问题
    prob.solve(solver=cp.ECOS)

    return n, throughput.value

# l = []
# for i in range(4, 17, 2):
#     x = get_result(i * i, get_torus(i), get_traffic_pattern_c(get_random_traffic_pattern(i), i))
#     print(x)
#     l.append(x)
# l
    

In [7]:
def get_ring(size):
    # 构造网络边
    e = []
    for i in range(size):
        left = (i - 1 + size) % size
        right = (i + 1) % size
        this = i
        to = [left, right]
        for y in to:
            e.append((this, y, 1.0))
    return e



ring_res = []
# for i in range(4, , 2):
i = 5
x = get_result(i, get_ring(i), get_traffic_pattern_c(get_random_traffic_pattern(i), i))
print(x)
ring_res.append(x)
ring_res

(5, 1.3333333306120365)


[(5, 1.3333333306120365)]

## 多商品流问题的约束条件如下：

1. **容量限制**：对于每一条边(u, v)，所有商品的流量之和不能超过这条边的容量。用数学公式表示就是：
   $$\sum_{i=1}^{k}f_{i}(u,v)\leq c(u,v)$$
   这里，$f_{i}(u,v)$表示商品$i$从节点$u$到节点$v$的流量，$c(u,v)$表示边(u, v)的容量。

2. **流守恒**：对于每一个商品$i$和每一个非源非汇节点$u$，进入节点$u$的流量之和等于离开节点$u$的流量之和。用数学公式表示就是：
   $$\sum_{q\in V}f_{i}(u,q)-\sum_{w\in V}f_{i}(w,u)=0\quad \mathrm {when} \quad u\neq s_{i},t_{i}$$
   这里，$s_{i}$和$t_{i}$分别表示商品$i$的源节点和汇节点。

3. **需求的满足**：对于每一个商品$i$，从源节点$s_{i}$出发的流量之和等于进入汇节点$t_{i}$的流量之和，也就是商品$i$的需求量$d_{i}$。用数学公式表示就是：
   $$\sum_{w\in V}f_{i}(s_{i},w)=d_{i}\Leftrightarrow \sum_{w\in V}f_{i}(w,t_{i})=d_{i}$$

这些约束条件确保了每一个商品的流量在网络中的传输不会超过任何一条边的容量，同时也满足了每一个商品的需求量。

## 问题对求解最大吞吐的适配
这里为了求解最大吞吐，我修改了这个约束条件，使得需求成为我要寻找的决策变量，即：

1. **容量限制**：对于每一条边(u, v)，所有传输流量之和不能超过这条边的容量。用数学公式表示就是：
   $$\sum_{i=1}^{k}f_{i}(u,v)\leq c(u,v)$$
   这里，$f_{i}(u,v)$表示商品$i$从节点$u$到节点$v$的流量，$c(u,v)$表示边(u, v)的容量。

2. **流守恒**：对于每一个传输$i$和每一个非源非汇节点$u$，进入节点$u$的流量之和等于离开节点$u$的流量之和。用数学公式表示就是：
   $$\sum_{q\in V}f_{i}(u,q)-\sum_{w\in V}f_{i}(w,u)=0\quad \mathrm {when} \quad u\neq s_{i},t_{i}$$
   这里，$s_{i}$和$t_{i}$分别表示商品$i$的源节点和汇节点。

3. **需求的满足**：对于每一个传输$i$，从源节点$s_{i}$出发的流量之和等于进入汇节点$t_{i}$的流量之和，也就是转移矩阵中每一个非对角线元素，记为$L_{s_i t_i}$的值乘以吞吐指标$d$。用数学公式表示就是：
   $$\sum_{w\in V}f_{i}(s_{i},w)=d * L_{s_i t_i}\Leftrightarrow \sum_{w\in V}f_{i}(w,t_{i})= d * L_{s_i t_i}$$

而目标函数就是: 
$$\max d$$